# 1. Prospect Overview
## 1.1. Intention
Basket/Spread option pricing.
## 1.2. Problem
For basket and spread options: There are more than one underlying asset.

For BSM model: Lognormal distribution. Lognormal RV sum is neither lognormal nor has analytic distribution.

For Normal model: Normal distribution. Normal RV sum is still normal and we can have analytic distribution.
## 1.3. Solution
We use Monte-Carlo simulation based on BSM model and control variate based on Normal model to price basket/spread options.
<img src = 'figure\1.PNG'>

# 2. Notice in Implementation
## 2.1. MC Pricing
There is no difficulty in MC pricing. We have already done this part in HW3.
## 2.2. Analytic Approximations
For BSM model: The unit of volatility is same as the rate of return of the asset.

For Normal model: The unit of volatility is same as the asset price.

Therefore, we need to transfer sigma(bsm) to sigma(normal).

In HW3, we simply use the following function:
<img src = 'figure\2.PNG'>
However, it is not accurate enough. We can improve the volatility transformation function.

### Consider only one asset
From what we learned in class, implied normal volatility varies with K for constant BSM volatility.
<img src = 'figure\3.PNG'>
Therefore, we can use normal volatility of point B instead of point A in the following figure to improve the accurancy of normal volatility.
<img src = 'figure\4.PNG'>
That is <img src = 'figure\5.PNG'>
Note that <img src = 'figure\6.PNG'>

### Consider multiple underlying assets
<img src = 'figure\7.PNG'>
Note that <img src = 'figure\8.PNG'>
Then we can calculate the overall volatility of the basket/spread option based on the following function.
<img src = 'figure\9.PNG'>
And finally we can use the normal model formula to calculate analytic approximations, just like HW3.

# 3. Basic Idea of the Project
The problem of the model above is that S(T) of all assets are RVs.

Only based on the fact that their weighted sum equals to K, we will have multiple solutions for S(T).

Our goal is to find S(T) of every asset with the largest probability based on the distribution of S(T).

# 4. Solution for S(T)
<img src = 'figure\0.PNG'>

We try to use Newton iteration method and simulation the solution step by step, hopefully we can find out the solutions!

# 5. Pros & Cons
Pros: 
   1. The calculation of volatility is more accurate;
   2. Weights of underlying assets can be positive or negative, which can be applied to both spread and basket options


Cons:
   1. The way to find S(T) is a little complex.

# 6. Function Test

In [2]:
import numpy as np
from option_models import basket
from option_models import basket_normal_volatility as bnv

In [2]:
### only run this when you changed the class definition
import imp
imp.reload(basket)
imp.reload(bnv)

<module 'option_models.basket_normal_volatility' from 'C:\\Users\\16012\\Documents\\Project-team_0001\\option_models\\basket_normal_volatility.py'>

In [3]:
# Function 1 in basket
# basket_price_mc(strike, spot, vol, weights, texp, cor_m, intr=0.0, divr=0.0, cp_sign=1, bsm=True, n_samples = 10000)
# This is a function for Monte-Carlo price based on BSM model & Normal model 

# example
spot = np.ones(4) * 100
vol = np.ones(4) * 0.2
weights = np.ones(4) * 0.25
divr = np.zeros(4)
intr = 0
cor_m = 0.0001*np.identity(4) + 0.9999*np.ones((4,4))
texp = 1
strike = 102

# For BSM model, we use BSM volitility and bsm=true.
p1 = basket.basket_price_mc(strike, spot, vol, weights, texp, cor_m, intr=intr, divr=divr, cp_sign=1, bsm=True, n_samples = 10000)

# For Normal model, we use Normal volitility = vol*spot and bsm=false.
p2 = basket.basket_price_mc(strike, spot, vol*spot, weights, texp, cor_m, intr=intr, divr=divr, cp_sign=1, bsm=False, n_samples = 10000)

print("mc_bsm\tmc_normal\n%.4f\t%.4f"%(p1,p2))

mc_bsm	mc_normal
7.0867	7.0399


In [4]:
# Function 2 in basket
# basket_price_norm_analytic(strike, spot, vol, weights, texp, cor_m, intr=0.0, divr=0.0, cp_sign=1)
# This is a function for analytic value based on Normal model 

# example
spot = np.ones(4) * 100
vol = np.ones(4) * 0.2
weights = np.ones(4) * 0.25
divr = np.zeros(4)
intr = 0
cor_m = 0.0001*np.identity(4) + 0.9999*np.ones((4,4))
texp = 1
strike = 102

# here volatility use the normal volatility = vol*spot
p1 = basket.basket_price_norm_analytic(strike, spot, vol*spot, weights, texp, cor_m, intr=intr, divr=divr, cp_sign=1)
print ("analytical_normal\n",p1)

analytical_normal
 7.01840890416


In [5]:
# function 1 in basket_normal_volatility
# basket_normal_volatility(strike, spot, vol, weights, texp, cor_m, intr=0.0, divr=0.0, cp_sign=1)
# This function compute the basket_normal_volatility by local volatility model, \
# that is, norm_impvol_k=0.5*(Sk_0+Sk_T)*bsm_norm_k

# example
spot = np.ones(4) * 100
vol = np.ones(4) * 0.2
weights1 = np.array([1, 0, 0, 0])
weights2 = np.array([0.1, 0.2, 0.3, 0.4])
divr = np.zeros(4)
intr = 0
cor_m = 0.8*np.identity(4) + 0.2*np.ones((4,4))
texp = 1
strike = 102

norm_vol1 = bnv.basket_normal_volatility(strike, spot, vol, weights1, texp, cor_m)
norm_vol2 = bnv.basket_normal_volatility(strike, spot, vol, weights2, texp, cor_m)

print(norm_vol1, '\n', norm_vol2)

[ 20.1999999   19.40180674  19.35859222  19.74608901] 
 [ 20.65854969  20.16748174  20.05615683  20.20950386]


In [16]:
# Function 3 in basket
# basket_price_mc_cv(strike, spot, vol, weights, texp, cor_m, intr=0.0, divr=0.0, cp_sign=1, norm_vol_method=1,n_samples=10000, seed=8888)
# This is a function for BSM model approximation for Control Variate.
# If norm_vol_method=1, then norm_vol_method = vol*spot
# If norm_vol_method=2, then norm_vol_method = our basket_normal_volatility